In [ ]:
# My OpenAI Key
import os
os.environ['OPENAI_API_KEY'] = "INSERT OPENAI KEY"

In [ ]:
from gpt_index.index import GPTIndex

In [ ]:
index = GPTIndex.from_input_dir('data')

In [ ]:
index.save_to_disk('index.json')

In [ ]:
# try loading
new_index = GPTIndex.load_from_disk('index.json')

In [ ]:
# try verbose=True for more detailed outputs
new_index.query("What did the author do during his time at Y Combinator?", verbose=True)